In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from sklearn.metrics import classification_report, f1_score
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import pickle
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


self.transitions = table of rules: which crayon can follow which.

self.start_transitions = which crayon is good to start.

self.end_transitions = which crayon is good to end.

reset_parameters = start with baby rules before learning real ones.

In [ ]:
class CRF(nn.Module):
    """Conditional Random Field layer for sequence labeling"""
    def __init__(self, num_tags, batch_first=True):
        super(CRF, self).__init__()
        self.num_tags = num_tags
        self.batch_first = batch_first

        # Transition parameters: transitions[i, j] is the score of transitioning from j to i
        self.transitions = nn.Parameter(torch.randn(num_tags, num_tags))

        # Start and end transitions
        self.start_transitions = nn.Parameter(torch.randn(num_tags))
        self.end_transitions = nn.Parameter(torch.randn(num_tags))
        self.reset_parameters()
    def reset_parameters(self):
        """Initialize parameters"""
        nn.init.uniform_(self.transitions, -0.1, 0.1)
        nn.init.uniform_(self.start_transitions, -0.1, 0.1)
        nn.init.uniform_(self.end_transitions, -0.1, 0.1)

total score of all possible cases (per o loc ..)

In [ ]:
def _compute_partition_function(self, emissions, mask):
        """Compute the partition function using forward algorithm"""
        #“emission scores” — how much each word (in a sentence) likes each possible tag. wth shape [batch_size, seq_length, num_tags]
        # and only takes the 2 frst
        batch_size, seq_length = emissions.size()[:2]

        # Initialize forward variables ( first word in each sentence + how easy it is to start a sequence with each tag. [1,numtags] )
        alpha = emissions[:, 0] + self.start_transitions.unsqueeze(0)

        for i in range(1, seq_length):
            # Broadcast alpha for all possible next tags
            #alpha has shape: [batch_size, num_tags]=>  [batch_size, num_tags,1] at position 2
            broadcast_alpha = alpha.unsqueeze(2)  # (batch_size, num_tags, 1)

            # Add transition scores If I’m in tag A, what’s the score of moving to tag B?
            broadcast_transitions = self.transitions.unsqueeze(0)  # (1, num_tags, num_tags)

            # Add emission scores
            broadcast_emissions = emissions[:, i].unsqueeze(1)  # (batch_size, 1, num_tags)

            # Compute next alpha
            next_alpha = broadcast_alpha + broadcast_transitions + broadcast_emissions

            # Log-sum-exp
            next_alpha = torch.logsumexp(next_alpha, dim=1)

            # Apply mask
            alpha = torch.where(mask[:, i].unsqueeze(1), next_alpha, alpha)

        # Add end transitions
        alpha = alpha + self.end_transitions.unsqueeze(0)

        # Sum over all tags
        return torch.logsumexp(alpha, dim=1)